# This notebook represents the training of the model at the core of the application

The training is performed on data collected from areas attacked by the beetle and areas where the attack was not recorded

In [12]:
import keras
import numpy as np
from sklearn.model_selection import train_test_split

In [29]:
NO_BARK_BEETLE = "dataset/no_bark_beetle_75.npy"
BARK_BEETLE = "dataset/bark_beetle_75.npy"

Loading the dataset

In [30]:
no_beetle_data = np.load(NO_BARK_BEETLE)
no_beetle_labels = np.zeros(no_beetle_data.shape[0])

beetle_data = np.load(BARK_BEETLE)
beetle_labels = np.ones(beetle_data.shape[0])


data = np.concatenate((beetle_data, no_beetle_data))
labels = np.concatenate((beetle_labels, no_beetle_labels))


In [31]:
labels = np.expand_dims(labels, axis=1)

Split the dataset between training and validation sets, to obtain some feedbacks on our work!

In [11]:
data

array([], dtype=float64)

In [32]:
X_train, X_val, y_train, y_val = train_test_split(data, labels, test_size=0.25, shuffle=True)

In [5]:
from sklearn.preprocessing import StandardScaler

In [33]:
X_train.shape

(74, 31, 75, 75)

In [34]:
X = X_train.reshape(74, 31*75*75)

In [19]:
print(X_train.shape)
print(X_val.shape)
print(y_train.shape)
print(y_val.shape)

(74, 31, 100, 100)
(25, 31, 100, 100)
(74, 1)
(25, 1)


In [35]:
seed_gen = keras.random.SeedGenerator(seed=21)
initializer = kernel_initializer=keras.initializers.RandomUniform(seed=seed_gen)

In [48]:
# create and fit the LSTM network
model = keras.Sequential()
model.add(keras.layers.Input(shape=(31, 75, 75)))
model.add(keras.layers.TimeDistributed(keras.layers.LSTM(128)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))

In [109]:
model.summary()

Model: "sequential_21"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ time_distributed_15             │ (None, 31, 128)        │       219,648 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_21 (Flatten)            │ (None, 3968)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_42 (Dense)                │ (None, 64)             │       254,016 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_43 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 473,729 (1.81 MB)

 Trainable params: 473,729 (1.81 MB)

 Non-trainable params: 0 (0.00 B)

In [49]:
metric = keras.metrics.BinaryAccuracy()
model.compile(
    optimizer=keras.optimizers.Adam(0.01), loss="binary_crossentropy", metrics=[metric]
)

In [50]:
checkpoint_filepath = 'model_75.keras'
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_binary_accuracy',
    mode='max',
    save_best_only=True)

In [51]:
model.fit(
    X_train,
    y_train,
    validation_data=(X_val, y_val),
    epochs=50,
    callbacks=[model_checkpoint_callback]
)

Epoch 1/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 26s 2s/step - binary_accuracy: 0.5427 - loss: 2.8803 - val_binary_accuracy: 0.5200 - val_loss: 0.8012
Epoch 2/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 559ms/step - binary_accuracy: 0.4865 - loss: 0.7702 - val_binary_accuracy: 0.5200 - val_loss: 0.6931
Epoch 3/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 548ms/step - binary_accuracy: 0.4456 - loss: 0.6932 - val_binary_accuracy: 0.4800 - val_loss: 0.6932
Epoch 4/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 554ms/step - binary_accuracy: 0.5174 - loss: 0.6931 - val_binary_accuracy: 0.4800 - val_loss: 0.6932
Epoch 5/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 535ms/step - binary_accuracy: 0.5213 - loss: 0.6931 - val_binary_accuracy: 0.4800 - val_loss: 0.6933
Epoch 6/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 535ms/step - binary_accuracy: 0.5018 - loss: 0.6931 - val_binary_accuracy: 0.4800 - val_loss: 0.6934
Epoch 7/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 542ms/step - binary_accuracy: 0.5096 - loss: 0.6930 - val_binary_accuracy: 0.4800 - val_loss: 0.6935
Epoch 8/50
3/3 ━━━━━━━